In [17]:
import pandas as pd
import joblib  # ← CAMBIADO: usar joblib para cargar

# Cargar datos y modelo
df = pd.read_csv("data_no_outliers.csv")

# Cargar modelo usando joblib
modelo_rf = joblib.load("modelo_random_forest.pkl")

print("Modelo Random Forest cargado correctamente:", type(modelo_rf))

Modelo Random Forest cargado correctamente: <class 'sklearn.ensemble._forest.RandomForestClassifier'>


In [18]:
# ============================================
# FASE 7: PREDICCIÓN INTERACTIVA DE RIESGO
# ============================================

import pandas as pd
import numpy as np

# --- Validar el modelo ---
print("Tipo del modelo cargado:", type(modelo_rf))

# --- DESCRIPCIÓN DE VALORES VÁLIDOS ---
print("""
Ingrese los datos de prueba según las siguientes recomendaciones:

- FamiliarPrimerGradoCC: True / False
- FamiliarSegundoGradoCC: True / False
- DiagnosticoPrevioCancer: True / False
- Ejercicio: "Ninguna", "<3 horas", "3-4 horas", ">4 horas"
- Alcohol: "Nunca", "Ocasional", "Frecuente", "Diario"
- Mamografia: True / False
- Menstruacion: "<12 anos", "12-13 anos", ">=14 anos"
- PrimerHijo: "No hijos", "Primer hijo <30", "Primer hijo >=30"
""")

# --- INPUT MANUAL ---
print("\nIngrese los datos de un paciente de prueba:")

id_paciente = int(input("id_paciente: "))
nombre = input("nombre: ")
rut = input("rut: ")
edad = int(input("edad: "))
sexo = input("sexo (M/F): ")
fecha_registro = input("fecha_registro (dd/mm/yyyy): ")
id_medico = int(input("id_medico: "))
FamiliarPrimerGradoCC = input("FamiliarPrimerGradoCC (True/False): ") == "True"
FamiliarSegundoGradoCC = input("FamiliarSegundoGradoCC (True/False): ") == "True"
DiagnosticoPrevioCancer = input("DiagnosticoPrevioCancer (True/False): ") == "True"
Ejercicio = input("Ejercicio (Ninguna/<3 horas/3-4 horas/>4 horas): ")
Alcohol = input("Alcohol (Nunca/Ocasional/Frecuente/Diario): ")
Mamografia = input("Mamografia (True/False): ") == "True"
Menstruacion = input("Menstruacion (<12 anos/12-13 anos/>=14 anos): ")
PrimerHijo = input("PrimerHijo (No hijos/Primer hijo <30/Primer hijo >=30): ")

# --- FUNCIÓN PARA CATEGORIZAR EDAD (IGUAL QUE EN LA FASE 2) ---
def categorizar_edad(edad):
    if pd.isnull(edad):
        return 2  # valor por defecto
    elif edad < 40:
        return 0
    elif edad < 50:
        return 1
    elif edad < 60:
        return 2
    else:
        return 3

# Categorizar la edad
edad_categorizada = categorizar_edad(edad)

# Crear DataFrame de prueba
df_prueba = pd.DataFrame([{
    'Edad': edad_categorizada,  # ← CAMBIADO: usar edad categorizada
    'FamiliarPrimerGradoCC': FamiliarPrimerGradoCC,
    'FamiliarSegundoGradoCC': FamiliarSegundoGradoCC,
    'DiagnosticoPrevioCancer': DiagnosticoPrevioCancer,
    'Ejercicio': Ejercicio,
    'Alcohol': Alcohol,
    'Mamografia': Mamografia,
    'Menstruacion': Menstruacion,
    'PrimerHijo': PrimerHijo
}])

# --- PREPROCESAMIENTO ---
map_actividad = {'Ninguna': 0, '<3 horas': 1, '3-4 horas': 2, '>4 horas': 3}
map_alcohol = {'Nunca': 0, 'Ocasional': 1, 'Frecuente': 2, 'Diario': 3}
map_menstruacion = {'<12 anos': 2, '12-13 anos': 1, '>=14 anos': 0}  # ← CORREGIDO: orden correcto
map_primerhijo = {'No hijos': 2, 'Primer hijo <30': 1, 'Primer hijo >=30': 0}  # ← CORREGIDO

df_prueba_encoded = df_prueba.copy()
df_prueba_encoded['Ejercicio'] = df_prueba_encoded['Ejercicio'].map(map_actividad)
df_prueba_encoded['Alcohol'] = df_prueba_encoded['Alcohol'].map(map_alcohol)
df_prueba_encoded['Menstruacion'] = df_prueba_encoded['Menstruacion'].map(map_menstruacion)
df_prueba_encoded['PrimerHijo'] = df_prueba_encoded['PrimerHijo'].map(map_primerhijo)

# Convertir booleanos a enteros
df_prueba_encoded['FamiliarPrimerGradoCC'] = df_prueba_encoded['FamiliarPrimerGradoCC'].astype(int)
df_prueba_encoded['FamiliarSegundoGradoCC'] = df_prueba_encoded['FamiliarSegundoGradoCC'].astype(int)
df_prueba_encoded['DiagnosticoPrevioCancer'] = df_prueba_encoded['DiagnosticoPrevioCancer'].astype(int)
df_prueba_encoded['Mamografia'] = df_prueba_encoded['Mamografia'].astype(int)

# --- Ajustar columnas al modelo entrenado ---
if hasattr(modelo_rf, 'feature_names_in_'):
    faltantes = [c for c in modelo_rf.feature_names_in_ if c not in df_prueba_encoded.columns]
    for c in faltantes:
        df_prueba_encoded[c] = 0
    df_prueba_encoded = df_prueba_encoded[modelo_rf.feature_names_in_]

# --- VERIFICAR QUE NO HAY NaN ---
print("\nVerificando datos antes de la predicción:")
print(df_prueba_encoded)
print("\nValores nulos:", df_prueba_encoded.isnull().sum().sum())

# Rellenar cualquier NaN restante con 0
df_prueba_encoded = df_prueba_encoded.fillna(0)

# --- PREDICCIÓN ---
y_pred_prueba = modelo_rf.predict(df_prueba_encoded)

# --- MAPEO DE RIESGO ---
mapa_riesgo = {
    0: "Bajo",
    1: "Moderado",
    2: "Alto"
}

riesgo_numerico = y_pred_prueba[0]
riesgo_texto = mapa_riesgo.get(riesgo_numerico, "Desconocido")

# --- RESULTADO FORMATEADO ---
print("\n" + "="*60)
print(f"RESULTADO DE LA PREDICCIÓN PARA: {nombre}")
print("="*60)
print(f"Tipo de Riesgo: {riesgo_numerico}")
print(f"Clasificación: {riesgo_texto}")
print("="*60)

Tipo del modelo cargado: <class 'sklearn.ensemble._forest.RandomForestClassifier'>

Ingrese los datos de prueba según las siguientes recomendaciones:

- FamiliarPrimerGradoCC: True / False
- FamiliarSegundoGradoCC: True / False
- DiagnosticoPrevioCancer: True / False
- Ejercicio: "Ninguna", "<3 horas", "3-4 horas", ">4 horas"
- Alcohol: "Nunca", "Ocasional", "Frecuente", "Diario"
- Mamografia: True / False
- Menstruacion: "<12 anos", "12-13 anos", ">=14 anos"
- PrimerHijo: "No hijos", "Primer hijo <30", "Primer hijo >=30"


Ingrese los datos de un paciente de prueba:

Verificando datos antes de la predicción:
   Edad  FamiliarPrimerGradoCC  FamiliarSegundoGradoCC  \
0     0                      1                       1   

   DiagnosticoPrevioCancer  Menstruacion  PrimerHijo  Ejercicio  Alcohol  \
0                        1             2           2          0        0   

   Mamografia  
0           1  

Valores nulos: 0

RESULTADO DE LA PREDICCIÓN PARA: Juana
Tipo de Riesgo: 2
Clasi

In [19]:
print(f"Total de registros: {len(df)}")

Total de registros: 292
